# 1

In [181]:
import pandas as pd
import numpy as np
import scipy.stats as stats


data = pd.read_csv('House_Price - House_Price.csv')

In [182]:
# пропущенные значения
data['n_hos_beds'] = data['n_hos_beds'].fillna(data['n_hos_beds'].median())
data['waterbody'] = data['waterbody'].fillna('None')

# One-hot кодирование
data_encoded = pd.get_dummies(data, columns=['airport', 'waterbody', 'bus_ter'])

# убираем признаки с низкой дисперсией и сильно коррелированные признаки
data_encoded = data_encoded.drop(columns=['parks', 'bus_ter_YES'])

# применяем логарифмирование к 'crime_rate' и 'resid_area'
data_encoded['log_crime_rate'] = np.log(data_encoded['crime_rate'] + 1)
data_encoded['log_resid_area'] = np.log(data_encoded['resid_area'] + 1)

# рассчитываем корреляцию между признаками
corr_matrix = data_encoded.corr()
threshold = 0.9
high_corr_vars = (corr_matrix.abs() > threshold) & (corr_matrix.abs() != 1)

# Удаляем сильно коррелированные признаки
high_corr_columns = [column for column in high_corr_vars.columns if any(high_corr_vars[column])]
data_encoded = data_encoded.drop(columns=high_corr_columns)

# 2

In [183]:
def significance_test(beta_hat, X, sigma_squared, s_index):
    # Матрица X^T X
    X_transpose_X = X.T @ X

    # Матрица ковариации коэффициентов beta (sigma^2 * (X^T X)^-1)
    covariance_matrix = sigma_squared * np.linalg.inv(X_transpose_X)

    # Дисперсия коэффициента beta s (с использованием матрицы ковариации)
    variance_beta_s = covariance_matrix[s_index, s_index]

    # Стандартизация коэффициента beta s
    standardized_beta_s = beta_hat[s_index] / np.sqrt(variance_beta_s)

    # Для двухстороннего теста, используем нормальное распределение
    p_value = 2 * (1 - stats.norm.cdf(np.abs(standardized_beta_s)))

    return standardized_beta_s, p_value

In [184]:
def linear_regression_with_test(target, features_scaled):

    # Добавляем столбец единичных значений для константы (intercept)
    X = np.c_[np.ones(features_scaled.shape[0]), features_scaled.values]  # Добавляем столбец с единицами в начало
    y = target.values.reshape(-1, 1)  # Преобразуем целевую переменную в столбец

    # Вычисление beta = (X^T X)^(-1) X^T y
    X_transpose = X.T
    try:
        XTX_inv = np.linalg.inv(X_transpose @ X)
        beta_hat = XTX_inv @ X_transpose @ y
    except np.linalg.LinAlgError:
        print("матрица X^T X вырождена")
        return None, None, None, None

    # предсказания
    y_hat = X @ beta_hat

    # ошибоки
    residuals = y - y_hat

    # дисперсия ошибок (sigma^2) = (sum epsilon_i^2) / (n - k - 1)
    n = X.shape[0]
    k = X.shape[1] - 1
    sigma_squared = np.sum(residuals**2) / (n - k - 1)

    # тест на значимость для каждого коэффициента
    results = []
    for s_index in range(beta_hat.shape[0]):
        standardized_beta_s, p_value = significance_test(beta_hat, X, sigma_squared, s_index)
        results.append((standardized_beta_s, p_value))

    return beta_hat, y_hat, residuals, sigma_squared, results



# стандартизация признаков
features = data_encoded
features_scaled = (features - features.mean()) / features.std()

target = 'price'
features_scaled.drop(columns=[target], inplace=True)
target = data_encoded[target]


beta_hat, y_hat, residuals, sigma_squared, significance_results = linear_regression_with_test(target, features_scaled)

print(f"Оценки коэффициентов линейной регрессии (β̂):\n{beta_hat}")
print(f"\nПредсказания ̂y (первые 5):\n{y_hat[:5]}")
print(f"\nОшибки (остатки) ̂epsilon (первые 5):\n{residuals[:5]}")
print(f"\nОценка дисперсии ошибок (sigma^2): {sigma_squared}")

Оценки коэффициентов линейной регрессии (β̂):
[[ 22.52885375]
 [ -1.40154139]
 [ -1.34463635]
 [  3.03260646]
 [  0.99281323]
 [  2.14315733]
 [ -4.08212953]
 [  0.70123141]
 [  0.17793811]
 [  0.20773393]
 [-51.24377274]
 [-46.766023  ]
 [-57.70509037]
 [-64.85546399]
 [  1.48586126]]

Предсказания ̂y (первые 5):
[[25.32778737]
 [27.15144437]
 [40.7186025 ]
 [32.20734834]
 [31.77948879]]

Ошибки (остатки) ̂ε (первые 5):
[[-1.32778737]
 [-5.55144437]
 [-6.0186025 ]
 [ 1.19265166]
 [ 4.42051121]]

Оценка дисперсии ошибок (σ^2): 43.8495873048517


# 3

In [185]:
for i, (standardized_beta_s, p_value) in enumerate(significance_results):
    print(f"\nКоэффициент beta_{i} стандартизированный: {standardized_beta_s}")
    print(f"p-value: {p_value}")


Коэффициент β̂_0 стандартизированный: [76.52997987]
p-value: [0.]

Коэффициент β̂_1 стандартизированный: [-2.56692821]
p-value: [0.01026039]

Коэффициент β̂_2 стандартизированный: [-2.3903407]
p-value: [0.01683275]

Коэффициент β̂_3 стандартизированный: [7.65178508]
p-value: [1.97619698e-14]

Коэффициент β̂_4 стандартизированный: [2.11354145]
p-value: [0.03455445]

Коэффициент β̂_5 стандартизированный: [6.15477757]
p-value: [7.51829488e-10]

Коэффициент β̂_6 стандартизированный: [-8.09374328]
p-value: [6.66133815e-16]

Коэффициент β̂_7 стандартизированный: [2.35291573]
p-value: [0.01862685]

Коэффициент β̂_8 стандартизированный: [0.60114105]
p-value: [0.54774605]

Коэффициент β̂_9 стандартизированный: [0.69557533]
p-value: [0.48669482]

Коэффициент β̂_10 стандартизированный: [-2.25204841e-06]
p-value: [0.9999982]

Коэффициент β̂_11 стандартизированный: [-2.32938236e-06]
p-value: [0.99999814]

Коэффициент β̂_12 стандартизированный: [-2.1656031e-06]
p-value: [0.99999827]

Коэффициент β̂

# 4

In [186]:
def backward_elimination(data, target, predictors, significance_level=0.05):
    data_encoded = pd.get_dummies(data, columns=[col for col in data.select_dtypes(include=['object']).columns])

    # Стандартизируем признаки
    features = data_encoded[predictors]
    features_scaled = (features - features.mean()) / features.std()

    # Выполняем начальную модель линейной регрессии
    target = data_encoded[target]
    beta_hat, y_hat, residuals, sigma_squared, significance_results = linear_regression_with_test(target, features_scaled)

    while True:
        # Находим переменную с наибольшим p-value
        max_p_value = max(significance_results, key=lambda x: x[1])[1]

        # Если p-value всех переменных меньше 0.05
        if max_p_value < significance_level:
            break

        # Иначе исключаем переменную с наибольшим p-value
        # Сначала получим индекс переменной с максимальным p-value
        max_p_index = max(range(len(significance_results)), key=lambda i: significance_results[i][1])

        # Извлекаем переменную с максимальным p-value
        excluded_variable = predictors[max_p_index-1]

        # Исключаем переменную с наибольшим p-value
        predictors.remove(excluded_variable)

        # Перезапускаем процесс с обновленным набором признаков
        beta_hat, y_hat, residuals, sigma_squared, significance_results = linear_regression_with_test(target, features_scaled[predictors])

    return predictors

In [187]:
target = 'price'
predictors = ['crime_rate', 'resid_area', 'room_num', 'age', 'n_hos_beds', 'airport_YES', 'waterbody_Lake']

significant_predictors = backward_elimination(data, target, predictors)

print(f"Значимые переменные после выполнения backward elimination:\n{significant_predictors}")

Значимые переменные после выполнения backward elimination:
['crime_rate', 'resid_area', 'room_num', 'age', 'n_hos_beds', 'airport_YES']
